## 0. Libarary 불러오기 및 경로설정

In [12]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [13]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

## 2. Test Dataset 정의

In [14]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

## 3. Inference

In [15]:
from facenet_pytorch import InceptionResnetV1

class InceptionResnet(InceptionResnetV1):
    def __init__(self, num_classes=18, pretrained='vggface2', classify=True):
        super().__init__()

class multilabel_dropout_IR(InceptionResnetV1):
    def __init__(self, hidden_size = 2, num_classes = 18, pretrained = 'vggface2', classify = True):
        super().__init__()
        self.hidden_size = hidden_size
        self.high_dropout = torch.nn.Dropout(1 / hidden_size)
        self.logits = torch.nn.Linear(512, num_classes)

    def forward(self, x):
        """Calculate embeddings or logits given a batch of input image tensors.
        Arguments:
            x {torch.tensor} -- Batch of image tensors representing faces.
        Returns:
            torch.tensor -- Batch of embedding vectors or multinomial logits.
        """
        x = self.conv2d_1a(x)
        x = self.conv2d_2a(x)
        x = self.conv2d_2b(x)
        x = self.maxpool_3a(x)
        x = self.conv2d_3b(x)
        x = self.conv2d_4a(x)
        x = self.conv2d_4b(x)
        x = self.repeat_1(x)
        x = self.mixed_6a(x)
        x = self.repeat_2(x)
        x = self.mixed_7a(x)
        x = self.repeat_3(x)
        x = self.block8(x)
        x = self.avgpool_1a(x)
        x = self.dropout(x)
        x = self.last_linear(x.view(x.shape[0], -1))
        x = self.last_bn(x)
        if self.classify:
            x = torch.mean(torch.stack([
            self.logits(self.high_dropout(x))
            for _ in range(self.hidden_size)
        ], dim=0), dim=0)
        else:
            x = F.normalize(x, p=2, dim=1)
        return x

model = InceptionResnet()
model.load_state_dict(torch.load("./model/InceptionResnet/best.pth"))

RuntimeError: Error(s) in loading state_dict for multilabel_dropout_IR:
	Missing key(s) in state_dict: "conv2d_1a.conv.weight", "conv2d_1a.bn.weight", "conv2d_1a.bn.bias", "conv2d_1a.bn.running_mean", "conv2d_1a.bn.running_var", "conv2d_2a.conv.weight", "conv2d_2a.bn.weight", "conv2d_2a.bn.bias", "conv2d_2a.bn.running_mean", "conv2d_2a.bn.running_var", "conv2d_2b.conv.weight", "conv2d_2b.bn.weight", "conv2d_2b.bn.bias", "conv2d_2b.bn.running_mean", "conv2d_2b.bn.running_var", "conv2d_3b.conv.weight", "conv2d_3b.bn.weight", "conv2d_3b.bn.bias", "conv2d_3b.bn.running_mean", "conv2d_3b.bn.running_var", "conv2d_4a.conv.weight", "conv2d_4a.bn.weight", "conv2d_4a.bn.bias", "conv2d_4a.bn.running_mean", "conv2d_4a.bn.running_var", "conv2d_4b.conv.weight", "conv2d_4b.bn.weight", "conv2d_4b.bn.bias", "conv2d_4b.bn.running_mean", "conv2d_4b.bn.running_var", "repeat_1.0.branch0.conv.weight", "repeat_1.0.branch0.bn.weight", "repeat_1.0.branch0.bn.bias", "repeat_1.0.branch0.bn.running_mean", "repeat

In [4]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

NameError: name 'MyModel' is not defined